# LangChain RAG Lab: Resume Q&A with Robo1! 📄🤖

Welcome to the LangChain RAG (Retrieval-Augmented Generation) Lab! In this notebook, you'll build a chatbot that answers interview questions using your resume. Robo1 is here to help—he loves finding answers in big piles of documents!

<table><tr>
<td><img src="images/robo1.png" alt="Robo1 - RAG Expert" width="120" /></td>
<td style="vertical-align:top; padding-left:20px;">
<b>Robo1 says:</b><br>
<i>"I can search your resume faster than you can say 'curriculum vitae'!"</i><br>
</td>
</tr></table>

Let's get started and see how RAG makes chatbots smarter!

# What is RAG (Retrieval-Augmented Generation)?
RAG combines the power of search (retrieval) with language models (generation). Instead of guessing, your chatbot can look up answers in documents—like your resume!

Robo1 loves RAG because it means less guessing and more knowing.

# Import Libraries and Setup
Let's import the necessary libraries and set up our environment for RAG-powered resume Q&A.

In [18]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment
openai_api_key = os.getenv("OPENAI_API_KEY")

# Load and Chunk Your Resume
Let's load your resume PDF and split it into smaller chunks so Robo1 can search it efficiently.

In [ ]:
pdf_path = "my_resume.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# Split resume into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

### Create a Vector Store and Retriever
We'll turn your resume chunks into searchable vectors so Robo1 can find the best answers fast.

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [ ]:
# Display information about the FAISS vector store
print("Number of documents in FAISS vectorstore:", len(vectorstore.docstore._dict))
print("Document IDs:", list(vectorstore.docstore._dict.keys())[:5])  # Show first 5 IDs


# Optionally, show a sample document stored in FAISS
sample_id = list(vectorstore.docstore._dict.keys())[0]
print("Sample document content:\n", vectorstore.docstore._dict[sample_id].page_content)

# Show vector representation details
print("FAISS index shape:", vectorstore.index.ntotal, "vectors,", vectorstore.index.d, "dimensions")
sample_vector = vectorstore.index.reconstruct(0)
print("Sample vector (first 10 values):", sample_vector[:10])

### Build the RAG Chatbot Chain
Now we'll connect everything together so Robo1 can answer your interview questions using your resume.

In [ ]:
llm = OpenAI(openai_api_key=openai_api_key)

prompt_template = """
You are answering interview questions based on the following resume context.
If the answer is not in the context, say \"I don't know based on my resume.\"
Context: {context}
Question: {question}
Answer as if you are the candidate:
"""

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": PromptTemplate.from_template(prompt_template)}
)

# Chat with Your Resume!
Ask interview questions and see how Robo1 answers using your resume. If he can't find the answer, he'll let you know!

In [27]:
# Ask your interview question here
question = "What are my technical skills?"
answer = qa_chain.invoke({"query": question})
print("Bot:", answer)

Bot: {'query': 'What are my technical skills?', 'result': '\nAnswer: Based on your resume, you have experience with AWS, Apache Spark, Kubernetes, and Airflow. You have also worked with advanced querying, data federation, and integration techniques, as well as GenAI and AI technologies such as LangChain and Hugging Face. Additionally, you have experience with migrating teams to the cloud on Kubernetes and ensuring best and secure practices.'}


---

<table><tr>
<td><img src="images/robo1.png" alt="Robo1 - RAG Expert" width="120" /></td>
<td style="vertical-align:top; padding-left:20px;">
<b>Robo1 says:</b><br>
<i>"Congratulations! You just built a RAG-powered chatbot. Now I can ace any interview—unless they ask about my favorite pizza topping!"</i><br>
<i>With RAG, your AI can find answers in documents faster than ever. 📄🤖</i>
</td>
</tr></table>

*Thanks for completing the LangChain RAG Lab!*